# Get Data for Analysis

In this file, we combine our exported Bloomberg data of ESG scores, financial metrics, and return with ESG ratings from Yahoo Finance. We also add in the S&P 500 index data to compare our portfolio returns to the market.

## Setup

In [1]:
import json
import numpy as np
import os
import pandas as pd
import re
import urllib.request
import yfinance as yf

In [2]:
# Set up paths
input_dir = 'inputs/'
output_dir = 'outputs/'

# Data files
esg_path = input_dir + 'Bloomberg_ESG.csv'
financial_path = input_dir + 'Bloomberg_Financial.csv'
esg_final_path = input_dir + 'Bloomberg_ESG_Final.csv'
financial_final_path = input_dir + 'Bloomberg_Financial_Final.csv'
joined_final_path = input_dir + 'Bloomberg_Joined_Final.csv'
yfinance_final_path = input_dir + 'Yahoo_Finance_ESG_Final.csv'

In [3]:
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

## Handle Bloomberg Data

From the Bloomberg Excel plugin, data is exported in a format that is difficult to work with.  We must pivot the data to use it in our analysis.

We start with two CSVs, *Bloomberg_Financial.csv* and *Bloomberg_ESG.csv* and combine them into one dataframe, *Bloomberg_ESG_Final.csv*.

In [4]:
# Create a function to pivot and clean the Bloomberg data format
def clean_bbg_path(path, num_unique_cols):
  # Read in the first row to grab companies
  with open(path, 'r') as f:
    tickers = f.readline()
  
  # Map numbers to tickers
  tickers = re.sub(r',+', ',', tickers).split(',')
  idx_to_tic = {i: tic for i, tic in enumerate(tickers)}

  # Load in dataset and remove residual columns
  df = pd.read_csv(path, skiprows=1)
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
  
  # Store stubnames and rename columns matching them
  stubnames = df.columns.tolist()[0:num_unique_cols]
  df.columns = ['.'.join([col, '0']) if i < num_unique_cols else col for i, col in enumerate(df.columns)]

  # Pivot
  df = df.reset_index()
  res_df = pd.wide_to_long(
      df, stubnames=stubnames, i=['index'], j='Ticker', sep='.')

  # Replace numbers in Ticker column with tickers
  res_df = res_df.reset_index()
  res_df['Ticker'] = res_df['Ticker'].map(idx_to_tic)

  # Replace " .NA." with na
  res_df = res_df.replace(' .NA.', np.nan)

  # Remove index column
  res_df = res_df.drop(columns=['index'])

  return res_df

In [5]:
# Gather ESG data
esgs = clean_bbg_path(esg_path, 20).dropna(axis=1, how='all')
esgs.to_csv(esg_final_path, index=False)

In [6]:
# Gather financial data
financials = clean_bbg_path(financial_path, 16).dropna(axis=1, how='all')
financials = financials.dropna(axis=0, subset=['Dates'])
financials.to_csv(financial_final_path, index=False)

C:\Users\nicol\AppData\Local\Temp\ipykernel_19824\1156676700.py:12: DtypeWarning: Columns (0,3,8,17,20,25,68,71,76,85,88,93,102,105,110,119,122,127,136,139,144,153,156,161,187,190,195,204,207,212,221,224,229,238,241,246,255,258,263,272,275,280,289,292,297,306,309,314,323,326,331,340,343,348,357,360,365,374,377,382,391,394,399,408,411,416,425,428,433,442,445,450,476,479,484,510,513,518,527,530,535,544,547,552,561,564,569,578,581,586,595,598,603,612,615,620,629,632,637,646,649,654,663,666,671,680,683,688,697,700,705,714,717,722,731,734,739,748,751,756,765,768,773,782,785,790,833,836,841,850,853,858,884,887,892,901,904,909,918,921,926,935,938,943,986,989,994,1003,1006,1011,1020,1023,1028,1037,1040,1045,1054,1057,1062,1071,1074,1079,1088,1091,1096,1105,1108,1113,1122,1125,1130,1139,1142,1147,1156,1159,1164,1173,1176,1181,1190,1193,1198,1207,1210,1215,1224,1227,1232,1241,1244,1249,1258,1261,1266,1275,1278,1283,1309,1312,1317,1326,1329,1334,1343,1346,1351,1360,1363,1368,1377,1380,1385,1394,1

In [7]:
# Now, we add columns for month and year only in ESG data for a proper join
esgs['Month'] = esgs['Dates'].str.split('/').str[0]
esgs['Year'] = esgs['Dates'].str.slice(start=-4)
financials['Month'] = financials['Dates'].str.split('/').str[0]
financials['Year'] = financials['Dates'].str.slice(start=-4)

# Drop rows that are not fully unique in financials, all are unique
financials = financials.drop_duplicates(['Ticker', 'Month', 'Year'])

In [8]:
# Now, merge the datasets and find any of those missing
merged_df = esgs.merge(
    financials.drop(columns=['Dates']),
    on=['Ticker', 'Month', 'Year'],
    how='outer',
    validate='1:1',
    indicator=True)

# Find tickers with missing esg data (Ticker when _merge == right_only)
missing_esg = merged_df.loc[
    merged_df['_merge'] == 'right_only', 'Ticker'].unique()
missing_esg = pd.DataFrame(missing_esg, columns=['Ticker'])

# Find tickers with missing financial data (Ticker when _merge == left_only)
missing_financial = merged_df.loc[
    merged_df['_merge'] == 'left_only', 'Ticker'].unique()
missing_financial = pd.DataFrame(missing_financial, columns=['Ticker'])

# Find tickers with both esg and financial data
both = merged_df.loc[merged_df['_merge'] == 'both', 'Ticker'].unique()
both = pd.DataFrame(both, columns=['Ticker'])

# Print
print(f'Number of tickers with missing esg data: {len(missing_esg)}')
print(f'Number of tickers with missing financial data: {len(missing_financial)}')
print(f'Number of tickers with both esg and financial data: {len(both)}')

# Show all rows
pd.set_option('display.max_rows', None)
print(missing_esg)


Number of tickers with missing esg data: 16
Number of tickers with missing financial data: 0
Number of tickers with both esg and financial data: 748
            Ticker
0    AVY UN Equity
1     GD UN Equity
2    HAS UW Equity
3    HSY UN Equity
4   ILMN UW Equity
5   INTC UW Equity
6    JNJ UN Equity
7    MHK UN Equity
8    NUE UN Equity
9    SNA UN Equity
10   TDY UN Equity
11   TER UW Equity
12   TMO UN Equity
13  TSCO UW Equity
14   TXT UN Equity
15   TER UN Equity


In [9]:
# Save
merged_df.drop(columns=['_merge']).to_csv(joined_final_path, index=False)

## Gather Yahoo! Finance ESG Scores

Now that we have our set, we will find the scores of all the companies in our set from Yahoo! Finance.  We use the `yfinance` library to get the data.

In [10]:
import pandas_datareader as pdr
from datetime import datetime
import yfinance as yf

Load all historical and current S&P 500 companies

In [11]:
historicalsp500 = pd.read_csv('inputs/historical_sp500.csv')
historicalsp500['Ticker'] = historicalsp500['Ticker'].str.split(' ').str[0]
historicalsp500['Date'] = pd.to_datetime(historicalsp500['Date'])

In [12]:
index_tickers = historicalsp500['Ticker'].tolist() #assigning all tickers from S&P 500 to a list
index_tickers = list(set(index_tickers))
#index_tickers3.pop(0)

In [13]:
dataframes = []

for symbol in index_tickers:
    url = f"https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={symbol}"
    connection = urllib.request.urlopen(url)
    data = connection.read()
    data_2 = json.loads(data)
    if "symbolSeries" in data_2["esgChart"]["result"][0]:
        Formatdata = data_2["esgChart"]["result"][0]["symbolSeries"]
        Formatdata_2 = pd.DataFrame(Formatdata)
        Formatdata_2["timestamp"] = pd.to_datetime(Formatdata_2["timestamp"], unit="s")
        Formatdata_2["ticker"] = symbol
        dataframes.append(Formatdata_2)
    #else:
        #print(f"No data found for {symbol}")

ALL_SP_ESG = pd.concat(dataframes)


In [ ]:
ALL_SP_ESG.columns = ['Date','yf_ESG_Score','yf_E_Score', 'yf_S_Score', 'yf_G_Score','Ticker']

In [ ]:
yfinanceESG = pd.merge(historicalsp500,ALL_SP_ESG,on=['Ticker','Date'],   how='outer',  
         # NOTICE! "how" has changed from prior example to outer
                validate='many_to_many',
                #indicator=True
        )#['_merge'].value_counts()          # ['_merge'].value_counts()  

In [ ]:
yfinanceESG.to_csv(yfinance_final_path, index=False)